# グラフ彩色問題のヒューリスティック手法

## 構築法

- seq_assignment: 与えられた点の番号の順に，彩色可能な最小の番号の色で塗っていく．
- largest_first: 点の次数（隣接する点の数）が大きいものから順に彩色する．
    - 次数が大きい方から使う理由は？
- dsatur: 構築法の途中で得た動的な情報をもとに，次に彩色する点を選択する．具体的には，次に彩色する点を， 隣接する点ですでに使われた色数が最大のもの（同点の場合には，彩色されていない隣接点の数が最大のもの）とする．
- recursive_largest_fit: 同じ色を塗ることができる点の集合（色クラス）は，互いに隣接していてはいけない． これは，安定集合に他ならない． 新しい色で彩色するための安定集合を順次求めることによって解を構築する．
    - 以下を繰り返す．次数(塗られていない隣接頂点数)が最も大きい頂点に新しい色を塗り，その頂点を含む大きな安定集合を貪欲に求める (次数が大きい順に？なぜ？)

In [1]:
import random
random.seed(42)

import graphtools as gts
from gcp_heur import seq_assignment, largest_first, dsatur, recursive_largest_fit
from gcp_heur import recursive_largest_fit_2

print("*** graph coloring problem ***")
print()

print("instance randomly created")
nodes, adj = gts.rnd_adj_fast(100, 0.5)

print("sequential assignment")
color, K = seq_assignment(nodes, adj)
print("solution: z =", K)
print(color)
print()
print("largest fit")
color, K = largest_first(nodes, adj)
print("solution: z =", K)
print(color)
print()
print("dsatur")
color, K = dsatur(nodes, adj)
print("solution: z =", K)
print(color)
print()
print("recursive largest fit")
color, K = recursive_largest_fit(nodes, adj)
print("solution: z =", K)
print(color)
print()
print("recursive largest fit (minimum ver)")
color, K = recursive_largest_fit_2(nodes, adj)
print("solution: z =", K)
print(color)
print()


*** graph coloring problem ***

instance randomly created
sequential assignment
solution: z = 20
[0, 0, 1, 1, 2, 0, 3, 4, 0, 2, 1, 3, 1, 5, 3, 1, 1, 2, 2, 4, 3, 4, 6, 4, 6, 5, 6, 3, 4, 0, 3, 5, 7, 8, 9, 7, 8, 7, 8, 1, 10, 6, 7, 2, 8, 9, 6, 9, 11, 9, 12, 12, 8, 11, 7, 7, 11, 11, 5, 5, 10, 4, 10, 2, 2, 13, 12, 14, 13, 15, 13, 9, 10, 14, 13, 15, 14, 14, 9, 14, 10, 16, 13, 12, 17, 16, 12, 17, 16, 17, 13, 14, 17, 15, 10, 18, 15, 18, 18, 19]

largest fit
solution: z = 20
[8, 11, 5, 14, 12, 7, 17, 0, 7, 4, 0, 18, 13, 6, 5, 15, 6, 4, 8, 13, 12, 6, 5, 12, 16, 2, 17, 3, 19, 8, 18, 13, 3, 16, 13, 8, 15, 13, 7, 14, 1, 10, 10, 16, 7, 9, 12, 15, 10, 0, 9, 12, 12, 3, 17, 11, 6, 18, 9, 2, 1, 8, 16, 18, 10, 3, 15, 14, 4, 0, 7, 5, 1, 2, 3, 3, 14, 4, 9, 9, 14, 10, 3, 16, 11, 6, 11, 7, 6, 19, 3, 17, 2, 4, 15, 5, 9, 11, 1, 5]

dsatur
solution: z = 18
[8, 11, 10, 17, 12, 1, 16, 14, 6, 3, 9, 13, 7, 2, 10, 16, 15, 6, 8, 14, 8, 11, 5, 14, 5, 2, 1, 4, 16, 8, 10, 9, 4, 16, 1, 3, 11, 9, 4, 17, 14, 12, 3, 6, 6, 7,

## タブーサーチ

彩色数 $K$ を固定し，実行可能解($K$彩色)を探索する．
彩色に対して，両端点が同色で塗られた辺の数を最小化し，0が得られればよい．

頂点 $i$ と同色の隣接頂点の数を $B(i)$ とおくと，目的関数は
$$
\sum_i B(i)/2
$$
と書ける．
タブーサーチでは，$B(i)>0$ となる $i$ の色を変更する操作を繰り返す(近傍の制限)．
タブーリストには，頂点と色のペアを保持する．
解の更新の際は，効率性のため，各頂点に対して「タブーでない色の変更のうち最も良い変更先の色」を常に保持し，最も良い頂点と色のペアを$O(n)$で取得する．解の更新の際に選ばれた頂点の隣接頂点で保持データを更新する．

In [2]:
random.seed(42)
from gcp_ts import rsatur, evaluate, tabu_search

nodes, adj = gts.rnd_adj_fast(100, 0.5)
K = 15  # tentative number of colors
print("tabu search, trying coloring with", K, "colors")
color = rsatur(nodes, adj, K)
print("starting solution: z =", evaluate(nodes, adj, color))
print("color:", color)
print()

print("starting tabu search")
tabulen = K
max_iter = 1000
color, sum_bad_degree = tabu_search(nodes, adj, K, color, tabulen, max_iter)
print("final solution: z =", sum_bad_degree)
print("color:", color)

tabu search, trying coloring with 15 colors
starting solution: z = 94
color: [10, 7, 11, 10, 9, 5, 2, 11, 1, 6, 14, 9, 11, 4, 7, 5, 0, 1, 7, 8, 5, 0, 2, 1, 5, 12, 2, 13, 4, 12, 3, 8, 12, 5, 10, 3, 9, 13, 9, 10, 9, 7, 13, 10, 1, 6, 13, 11, 11, 3, 5, 2, 9, 0, 10, 3, 12, 3, 10, 3, 7, 2, 4, 6, 11, 8, 0, 8, 14, 14, 7, 11, 12, 13, 11, 0, 0, 12, 10, 14, 7, 6, 4, 8, 14, 0, 2, 4, 4, 5, 6, 14, 3, 8, 7, 1, 13, 1, 5, 10]

starting tabu search
final solution: z = 4
color: [10, 2, 6, 6, 12, 9, 14, 0, 7, 12, 0, 13, 11, 4, 7, 14, 4, 12, 7, 4, 10, 4, 1, 6, 5, 6, 1, 13, 4, 1, 6, 5, 14, 7, 10, 14, 4, 5, 9, 11, 9, 5, 8, 14, 9, 3, 5, 11, 11, 3, 13, 2, 10, 11, 3, 3, 1, 14, 10, 3, 7, 1, 7, 12, 13, 8, 0, 2, 12, 0, 5, 11, 13, 2, 11, 8, 2, 12, 10, 10, 7, 6, 8, 8, 4, 13, 3, 9, 4, 5, 13, 2, 3, 8, 0, 6, 9, 12, 14, 1]


## 遺伝的アルゴリズムとタブーサーチの融合法

複数の解を保持しておき，交叉(crossover)とタブーサーチを繰り返して新しい解候補に置き換えていく．

1. 初期解を $n_\mathrm{elem}$ 個生成する．
2. 解をランクづけし，$i$ 位の解を $n_\mathrm{elem}-i+1$ に比例する確率で二つ選ぶ．
3. 選ばれた二つの解の子となる解を作る．
    1. 片方の親の解が分割した頂点集合たちの中から頂点数が最大のものを選ぶ
    2. その頂点集合の各頂点をその色で塗る (すでに塗られていたらスキップする)
    3. 2で塗った頂点集合を両方の親の解から除く．
    4. 1-3 を，親を交代しながら繰り返す
    5. 子の塗られていない頂点をランダムに塗る
4. 子の解を初期解とするタブーサーチを実行して，得られた解を解候補に追加し，最も悪い解を一つ除く．
5. 2-4 を繰り返す．


In [3]:
random.seed(42)
from gcp_ga import gcp_ga

nodes, adj = gts.rnd_adj_fast(100, 0.5)
K = 15  # tentative number of colors
print(
    "genetic algorithm (intensification with tabu search), trying coloring with",
    K,
    "colors",
)
color = rsatur(nodes, adj, K)
print("starting solution: z =", evaluate(nodes, adj, color))
print("color:", color)
print()

print("starting evolution with genetic algorithm")
nelem = 10
ngen = 100
TABULEN = K
TABUITER = 100
color, sum_bad_degree = gcp_ga(nodes, adj, K, ngen, nelem, TABULEN, TABUITER)
print("final solution: z =", sum_bad_degree)
print("color:", color)

genetic algorithm (intensification with tabu search), trying coloring with 15 colors
starting solution: z = 94
color: [10, 7, 11, 10, 9, 5, 2, 11, 1, 6, 14, 9, 11, 4, 7, 5, 0, 1, 7, 8, 5, 0, 2, 1, 5, 12, 2, 13, 4, 12, 3, 8, 12, 5, 10, 3, 9, 13, 9, 10, 9, 7, 13, 10, 1, 6, 13, 11, 11, 3, 5, 2, 9, 0, 10, 3, 12, 3, 10, 3, 7, 2, 4, 6, 11, 8, 0, 8, 14, 14, 7, 11, 12, 13, 11, 0, 0, 12, 10, 14, 7, 6, 4, 8, 14, 0, 2, 4, 4, 5, 6, 14, 3, 8, 7, 1, 13, 1, 5, 10]

starting evolution with genetic algorithm

xover of  5 and 2 	--> [0, 6, 7, 10, 12, 0, 6, 3, 1, 13, 10, 14, 7, 2, 11, 2, 2, 8, 12, 4, 3, 11, 0, 1, 1, 1, 6, 14, 4, 1, 10, 14, 14, 11, 7, 4, 12, 4, 13, 4, 6, 8, 8, 3, 10, 13, 5, 13, 7, 9, 3, 8, 7, 0, 4, 8, 2, 8, 5, 2, 11, 5, 3, 3, 8, 12, 14, 5, 12, 0, 12, 7, 1, 1, 0, 14, 0, 2, 8, 7, 8, 2, 12, 2, 12, 2, 1, 6, 1, 4, 14, 0, 4, 13, 13, 2, 14, 9, 6, 4]
mutate (tabu search) 	--> [0, 6, 7, 10, 12, 0, 6, 3, 1, 13, 10, 14, 7, 2, 11, 2, 2, 8, 12, 4, 3, 11, 0, 1, 1, 1, 6, 14, 4, 1, 10, 14, 14, 11, 7, 4, 